In [1]:
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
import math 

class Whisper_gallery_resonator( QComponent ):
    """ A double chip resonator devic "vrunda_design .ipynb"e "
    default_options = ( radius = pass,
                        outer_cir_pos_x = ,
                        outer_cir_pos_y = ,
                        transmon_margin = ,
                        trans_rect_width = ,
                        trans_rect_len = ,
                        trans_rect_pos_x = ,
                        trans_rect_pos_y = ,
                        inner_cir_pos_x = ,
                        inner_cir_pos_y = ,
                        layer = ,
                        chip = ,
                    )"""
    component_metadata = Dict(short_name='WGR',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""
    default_options = dict( radius = '7.5mm',
                            outer_cir_pos_x = '0mm',
                            outer_cir_pos_y = '0mm',
                            inner_cir_rotation = '45',
                            inner_cir_radius = '5mm',
                            transmon_margin = '0.05mm',
                            trans_orientation = '90',
                            trans_qubit_rotation = '0',
                            trans_rect_height = '0.23mm',
                            trans_rect_width = '0.5mm',
                            trans_rect_pos_x = '7.35mm',
                            trans_rect_pos_y = '0mm',
                            trans_pad_width = '0.45mm',
                            trans_pad_height = '0.12mm',
                            epsilon = '0.05um',
                            layer = '1',
                            chip = 'main')
    def make( self):
        p = self.p
        ################################## defining outer_circle #################################################################
        outer_circle = draw.shapely.geometry.Point(p.outer_cir_pos_x, p.outer_cir_pos_y).buffer( p.radius)

        #################################### defining transmon ###################################################################                                                                                     
        r_transmon = p.outer_cir_pos_x + p.radius - p.transmon_margin - p.trans_rect_width/2
        pos_x = (r_transmon) * math.cos( p.trans_orientation * math.pi / 180 ) 
        pos_y = (r_transmon) * math.sin( p.trans_orientation * math.pi / 180 ) 
        r1 = draw.rectangle( p.trans_rect_width, p.trans_rect_height,pos_x, pos_y)
        r2 = draw.rectangle( p.trans_pad_width, p.trans_pad_height, pos_x, pos_y)
        transmon = draw.subtract( r1, r2)
        # trans_rotation= draw.rotate( trans, p.trans_qubit_rotation, origin = ( pos_x, pos_y))
        ################################### josephson_junction ###################################################################
        inductor_width = p.trans_rect_width - p.trans_pad_width
        ################################## condition for inner circle #############################################################
        rt = p.outer_cir_pos_x + p.radius - (2*p.transmon_margin)-p.trans_rect_width
        inner_cir_pos_x = (rt-p.inner_cir_radius) * math.cos( p.inner_cir_rotation * math.pi / 180 ) 
        inner_cir_pos_y = (rt-p.inner_cir_radius ) * math.sin( p.inner_cir_rotation * math.pi / 180 ) 
        if( math.sqrt(inner_cir_pos_x**2 + inner_cir_pos_y**2) + p.inner_cir_radius > rt):
            return False
        ''' **************************** Nearest semi-intermediate circle coordinates *************************************************
        # x_nearest = rt* math.cos( p.inner_cir_rotation * math.pi / 180 )
        # y_nearest = rt* math.sin( p.inner_cir_rotation * math.pi / 180 )
        # ri = math.sqrt( p.inner_cir_pos_x**2 + p.inner_cir_pos_y**2)
        # if ri>=rt:
        #     return False
        # inner_cir_radius = rt-ri
        ## defining inner circle 
        # inner_radius = p.radius -  (2* p.transmon_margin + p.trans_rect_len)-( p.outer_cir_pos_x - p.inner_cir_pos_x)'''
        ################################### inner_circle ##########################################################################
        inner_circle = draw.shapely.geometry.Point( inner_cir_pos_x, inner_cir_pos_y).buffer( p.inner_cir_radius )  

        #################################### phi ##################################################################################
        diff_rect_pad_width = (p.trans_rect_width - p.trans_pad_width)/2
        R1 = r_transmon - p.trans_pad_width/2  - diff_rect_pad_width/2
        a = diff_rect_pad_width/2 + p.trans_pad_width/2
        # phi = math.acos( (R1**2 + r_transmon**2 - a**2) / 2 *r_transmon*R1) + (p.trans_orientation * 180)/math.pi
        
        #################################### josephson_junction ######################################################################
        '''
        r_1 = R1 - diff_rect_pad_width/2
        r_2 = R1 + diff_rect_pad_width/2
        # R1_x = R1 * math.cos( p.trans_orientation * math.pi / 180 ) 
        # R1_y = R1 * math.sin( p.trans_orientation * math.pi /180 )
        r_1_x = r_1 * math.cos(p.trans_orientation * math.pi / 180 ) 
        r_1_y = r_1 * math.sin(p.trans_orientation  * math.pi / 180 ) 
        r_2_x = r_2 * math.cos(p.trans_orientation  * math.pi / 180 ) 
        r_2_y = r_2 * math.sin(p.trans_orientation  * math.pi / 180 ) 

        joseph = draw.LineString([(r_2_x, r_2_y),(r_1_x, r_1_y)])'''
        y1 = p.outer_cir_pos_y + p.trans_pad_height/2 - p.epsilon
        y2 = y1 + (p.trans_rect_height - p.trans_pad_height)/2 + p.epsilon 
        x1 = r_transmon
        joseph = draw.LineString([(x1,y1),(x1, y2)])

        
        final_design_1 = draw.subtract( outer_circle, r1)
        final_design_2 = draw.union(final_design_1 , r2)
        final_design_3 = draw.subtract( final_design_2, inner_circle)
        objects = [final_design_3,joseph]
        objects = draw.rotate(objects, p.orientation, origin=(p.outer_cir_pos_x, p.outer_cir_pos_y))
        objects = draw.translate(objects, p.pos_x, p.pos_y)
        [final_design_3,joseph] = objects
        self.add_qgeometry('poly', {'final_design_3': final_design_3},
                           subtract=True,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)
        self.add_qgeometry('junction', {'joseph': joseph},
                           subtract=True,
                           width= diff_rect_pad_width,
                           layer=p.layer,
                           chip=p.chip)
        
from shapely.geometry import CAP_STYLE
from qiskit_metal import draw, Dict  # , QComponent
from qiskit_metal.qlibrary.core import QComponent 
import math 

class Whisper_gallery_resonator_u( QComponent ):
    """ A double chip resonator device "
    default_options = ( radius = pass,
                        outer_cir_pos_x = ,
                        outer_cir_pos_y = ,
                        transmon_margin = ,
                        trans_rect_width = ,
                        trans_rect_len = ,
                        trans_rect_pos_x = ,
                        trans_rect_pos_y = ,
                        inner_cir_pos_x = ,
                        inner_cir_pos_y = ,
                        layer = ,
                        chip = ,
                    )"""
    component_metadata = Dict(short_name='WGR',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""
    default_options = dict( radius = '7.5mm',
                            outer_cir_pos_x = '0mm',
                            outer_cir_pos_y = '0mm',
                            inner_cir_rotation = '45',
                            inner_cir_radius = '5mm',
                            transmon_margin = '0.05mm',
                            # trans_orientation = '90',
                            # trans_qubit_rotation = '0',
                            trans_rect_len = '0.5mm',
                            # trans_rect_width = '0.23mm',
                            # trans_rect_pos_x = '7.35mm',
                            # trans_rect_pos_y = '0mm',
                            # trans_pad_width = '0.12mm',
                            # trans_pad_len = '0.45mm',
                            layer = '1',
                            chip = 'main')
    def make( self):
        p = self.p
        ################################## defining outer_circle #################################################################
        outer_circle = draw.shapely.geometry.Point(p.outer_cir_pos_x, p.outer_cir_pos_y).buffer( p.radius)

        #################################### defining transmon ###################################################################                                                                                     
        # r_transmon = p.outer_cir_pos_x + p.radius - p.transmon_margin - p.trans_rect_len/2
        # pos_x = (r_transmon) * math.cos( p.trans_orientation * math.pi / 180 ) 
        # pos_y = (r_transmon) * math.sin( p.trans_orientation * math.pi / 180 ) 
        # r1 = draw.rectangle( p.trans_rect_width, p.trans_rect_len,pos_x, pos_y)
        # r2 = draw.rectangle( p.trans_pad_width, p.trans_pad_len, pos_x, pos_y)
        # transmon = draw.subtract( r1, r2)
        # trans_rotation= draw.rotate( trans, p.trans_qubit_rotation, origin = ( pos_x, pos_y))
        ################################### josephson_junction ###################################################################
        # inductor_width = p.trans_rect_width - p.trans_pad_width
        ################################## condition for inner circle #############################################################
        rt = p.outer_cir_pos_x + p.radius - (2*p.transmon_margin)-p.trans_rect_len
        inner_cir_pos_x = (rt-p.inner_cir_radius) * math.cos( p.inner_cir_rotation * math.pi / 180 ) 
        inner_cir_pos_y = (rt-p.inner_cir_radius ) * math.sin( p.inner_cir_rotation * math.pi / 180 ) 
        if( math.sqrt(inner_cir_pos_x**2 + inner_cir_pos_y**2) + p.inner_cir_radius > rt):
            return False
        ''' **************************** Nearest semi-intermediate circle coordinates *************************************************
        # x_nearest = rt* math.cos( p.inner_cir_rotation * math.pi / 180 )
        # y_nearest = rt* math.sin( p.inner_cir_rotation * math.pi / 180 )
        # ri = math.sqrt( p.inner_cir_pos_x**2 + p.inner_cir_pos_y**2)
        # if ri>=rt:
        #     return False
        # inner_cir_radius = rt-ri
        ## defining inner circle 
        # inner_radius = p.radius -  (2* p.transmon_margin + p.trans_rect_len)-( p.outer_cir_pos_x - p.inner_cir_pos_x)'''
        ################################### inner_circle ##########################################################################
        inner_circle = draw.shapely.geometry.Point( inner_cir_pos_x, inner_cir_pos_y).buffer( p.inner_cir_radius )  
        # final_design_1 = draw.subtract( outer_circle, r1)
        # final_design_2 = draw.union(final_design_1 , r2)
        final_design_3 = draw.subtract( outer_circle, inner_circle)
        objects = [final_design_3]
        objects = draw.rotate(objects, p.orientation, origin=(p.outer_cir_pos_x, p.outer_cir_pos_y))
        objects = draw.translate(objects, p.pos_x, p.pos_y)
        [final_design_3] = objects
        self.add_qgeometry('poly', {'final_design_3': final_design_3},
                           subtract=True,
                           helper=p.helper,
                           layer=p.layer,
                           chip=p.chip)


C:\Users\uday mathur\anaconda3\envs\Qiskit_metal\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.12.0-CAPI-1.18.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
c:\users\uday mathur\qiskit-metal\qiskit_metal\__init__.py:30: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapel

In [2]:
from qiskit_metal import designs, MetalGUI
design  = designs.DesignFlipChip()
gui = MetalGUI( design ) 
gui.overwrite_enabled = True


design.overwrite_enabled = True
# Render the Quantum_Chip
q1 =  Whisper_gallery_resonator( design, "Q1", options = Dict(inner_cir_rotation = '0', trans_orientation = '0',chip = 'Q_chip', epsilon = '0.5um') )
gui.overwrite_enabled = True 

# Render the Contraol_Chip 
q2  =  Whisper_gallery_resonator_u( design, 'Q2', options = Dict( inner_cir_rotation = '0', trans_orientation = '0',chip = 'C_chip'))
gui.rebuild()
gui.autoscale()

01:27AM 39s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`
01:27AM 40s ERROR [__getitem__]: 
 Component name = `Q2`
 Option name    = `helper`
01:27AM 40s ERROR [__getitem__]: 
 Component name = `Q1`
 Option name    = `helper`
01:27AM 40s ERROR [__getitem__]: 
 Component name = `Q2`
 Option name    = `helper`


In [ ]:
# EPR ANALYSIS 
from qiskit_metal.analyses.quantization import EPRanalysis
import pyEPR as epr
eig_res = EPRanalysis(design, "hfss")

hfss.activate_ansys_design("Flip_Chip", 'eigenmode')
hfss.render_design()

# tuning parameters 
eig_res.sim.setup.max_passes = 6
eig_res.sim.setup.n_modes = 3
eig_res.sim.setup.vars.Lj = '11 nH'
eig_res.sim.setup.vars.Cj = '5 fF'

# Calling Ansys HFSS 
eig_res.sim.run()

# Describing Object "pinfo"
pinfo = epr.ProjectInfo( project_path = r'C:\Users\uday mathur\Desktop\qiskit metal jupyter\qiskit_metal_practice_31\EPR_project_files',
                        project_name = 'FLIP_CHIP_8',
                        design_name = 'Flip_Chip')

import warnings
warnings.filterwarnings("ignore")
# Start EPR running process 
eig_res.epr_start()
eig_res.run_epr()
# extracting all reslts for epr analysis 
eig_res.results